In [ ]:
import os
import numpy as np
from scipy.io import loadmat
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, WhiteKernel
from sklearn.metrics import mean_squared_error, make_scorer
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pickle

In [ ]:
dataset_size = 4000
test_size = 0.3
nb_test_points = round(dataset_size * test_size)

## Load data from mat

In [ ]:
mat = loadmat('../mat_files/orange_straight_0503.mat')
data = mat['straight_cuts'][0]
train_set, test_set = train_test_split(data, test_size=test_size, random_state=42)

downsampled_train_set = []
for i in range(len(train_set)):
    tmp = resample(train_set[i], n_samples=round((dataset_size-nb_test_points)/len(train_set)), replace=False, random_state=42)
    downsampled_train_set.append(tmp)

downsampled_test_set = []    
for i in range(len(test_set)):
    tmp = resample(test_set[i], n_samples=round(nb_test_points/len(test_set)), replace=False, random_state=42)
    downsampled_test_set.append(tmp)

#print(downsampled_train_set)

downsampled_train_set = np.concatenate(downsampled_train_set, axis=0)
downsampled_train_set = np.delete(downsampled_train_set,np.where(downsampled_train_set[:,1] < 0.009), axis=0)
#downsampled_train_set[:,2] = [10*x for x in downsampled_train_set[:,2]]
X_train = downsampled_train_set[:,2:4]
y_train = downsampled_train_set[:,4].reshape(-1, 1)
#print(X_train)
#print(y_train)
downsampled_test_set = np.concatenate(downsampled_test_set, axis=0)
downsampled_train_set = np.delete(downsampled_train_set,np.where(downsampled_train_set[:,1] < 0.009), axis=0)
#downsampled_test_set[:,2] = [10*x for x in downsampled_test_set[:,2]]
X_test = downsampled_test_set[:,2:4]
y_test = downsampled_test_set[:,4].reshape(-1, 1)
#X[:,0] = [-10*x for x in X[:,0]]

## Normalize data

In [ ]:
scaler = StandardScaler()
y_train_norm =scaler.fit_transform(y_train)
y_test_norm = scaler.transform(y_test)

## GPR

In [ ]:
#kernel = RBF([8.02e-05, 0.0012], length_scale_bounds='fixed') + WhiteKernel(0.0458, noise_level_bounds='fixed') # 2000, 0.3, aniso
kernel = RBF([0.0015, 0.003], length_scale_bounds='fixed') + WhiteKernel(0.3, noise_level_bounds='fixed') # 2000, 0.3
#kernel = RBF([0.01, 0.01]) + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
                               normalize_y=False,
                               n_restarts_optimizer=10,
                               random_state=42)
gpr.fit(X_train, y_train_norm)


In [ ]:
with open('rbf_orange_0503.pickle', 'wb') as f:
    pickle.dump({'gpr': gpr, 'scaler': scaler}, f)

In [ ]:
#kernel = Matern([0.00312, 0.00363], length_scale_bounds='fixed', nu=1.5) + WhiteKernel(0.0307, noise_level_bounds='fixed') # 2000, 0.3, aniso
kernel = Matern([0.02, 0.01], length_scale_bounds='fixed', nu=1.5) + WhiteKernel(0.01, noise_level_bounds='fixed') # 2000, 0.3, aniso
#kernel = Matern([0.01, 0.01], nu=1.5) + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
                               normalize_y=False,
                               n_restarts_optimizer=10,
                               random_state=42)
gpr.fit(X_train, y_train_norm)

In [ ]:
#kernel = Matern([0.00246, 0.00287], length_scale_bounds='fixed', nu=2.5) + WhiteKernel(0.0369, noise_level_bounds='fixed') # 2000, 0.3, aniso
kernel = Matern([0.025, 0.015], length_scale_bounds='fixed', nu=2.5) + WhiteKernel(0.01, noise_level_bounds='fixed') # 2000, 0.3, aniso
#kernel = Matern([0.01, 0.01], nu=2.5) + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
                               normalize_y=False,
                               n_restarts_optimizer=10,
                               random_state=42)
gpr.fit(X_train, y_train_norm)

In [ ]:
#print(gpr.get_params())
print(gpr.kernel_)

## Plot

In [ ]:
X1, X2 = np.meshgrid(np.linspace(np.min(X_train[:,0]),np.max(X_train[:,0]),100),np.linspace(np.min(X_train[:,1]),np.max(X_train[:,1]),100))
Y = np.zeros((100,100))
#Y_upper = np.zeros((100,100))
#Y_lower = np.zeros((100,100))
std_mesh = np.zeros((100,100))
for i in range(100):
    for j in range(100):
        res, std = gpr.predict(np.array([X1[i,j], X2[i,j]]).reshape(1,-1), return_std=True)
        Y[i,j] = scaler.inverse_transform(res)
        #Y_upper[i,j] = Y[i,j] + 2*np.sqrt(std) * scaler.scale_
        #Y_lower[i,j] = Y[i,j] - 2*np.sqrt(std) * scaler.scale_
        std_mesh[i,j] = 2 * std * scaler.scale_

Y_upper = Y + std_mesh
Y_lower = Y - std_mesh
        
        
#X1, X2 = np.meshgrid(np.sort(X_test[:,0]), np.sort(X_test[:,1]))
#Y = np.zeros(X1.shape)
#Y_upper = np.zeros(X1.shape)
#Y_lower = np.zeros(X1.shape)
#for i in range(Y.shape[0]):
#    for j in range(Y.shape[1]):
#        res, std = gpr.predict(np.array([X1[i,j], X2[i,j]]).reshape(1,-1), return_std=True)
#        Y[i,j] = scaler.inverse_transform(res)
#        Y_upper[i,j] = Y[i,j] + 2*np.sqrt(std) * scaler.scale_
#        Y_lower[i,j] = Y[i,j] - 2*np.sqrt(std) * scaler.scale_

fig = go.Figure(data=[
    go.Surface(z=Y, x=X1, y=X2, cmax=np.max(Y_upper), cmin=np.min(Y_lower)),
    go.Scatter3d(x=X_train[:,0].ravel(), y=X_train[:,1].ravel(), z=y_train.ravel(), mode='markers',
                 marker=dict(opacity=1, size=1, color='black')),
    go.Surface(z=Y_upper, x=X1, y=X2, cmax=np.max(Y_upper), cmin=np.min(Y_lower), showscale=True, opacity=0.8),
    go.Surface(z=Y_lower, x=X1, y=X2, cmax=np.max(Y_upper), cmin=np.min(Y_lower), showscale=True, opacity=0.8),
    ])
fig.update_layout(scene = dict(
                    xaxis_title='Depth [m]',
                    yaxis_title='Linear Velocity [m/s]',
                    zaxis_title='Force [N]'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
fig.show()

In [ ]:
from scipy.io import savemat
mdic = {"depth": X1, "vel": X2, "mean": Y, "std": std_mesh}
savemat("model.mat", mdic)
mdic = {"depth": X_train[:,0].ravel(), "vel": X_train[:,1].ravel(), "force": y_train.ravel()}
savemat("train_data.mat", mdic)